In [1]:
import warnings
warnings.filterwarnings('ignore')

import json
import os
import string
import time

import pandas as pd
import numpy as np

from pathlib import Path

In [2]:
json_dir = Path(r"../data/dvlog_text")
annotations_file = Path(r"../DVlog/dataset/dvlog_labels_v2.csv")
save_dir = Path(r"../DVlog/dataset/PDEM/")

In [3]:
# load in the annotation labels
df_annotations = pd.read_csv(annotations_file)
df_annotations.head()

,video_id,label,gender,dataset
0,0,1,f,train
1,1,1,f,test
2,2,1,m,train
3,3,1,m,train
4,4,1,f,test


In [4]:
save_dict = {
    "train": "training",
    "test": "test",
    "val": "validation"
}

# loop over each text file and build the transcript files
for json_file in os.listdir(json_dir):
    
    # get the video_id and setup the path to the file
    filename, _ = os.path.splitext(json_file)
    video_id = int(filename.split("_")[0])
    json_path = os.path.join(json_dir, json_file)
    
    with open(json_path) as current_file:
        json_dict = json.loads(current_file.read())


    # extract the sentences with their begin and end timestamp
    video_sents = []
    for sent_dict in json_dict.get("segments"):
        video_sents.append((sent_dict.get("start"), sent_dict.get("end"), sent_dict.get("text")))

    # save as dataframe
    df = pd.DataFrame(video_sents)

    # check in which folder to save
    current_sample = df_annotations[df_annotations["video_id"] == video_id]
    save_folder = save_dict.get(current_sample.dataset.values[0])

    save_file = os.path.join(save_dir, save_folder, f"{filename}.csv")

    df.to_csv(save_file, index=False, sep=';')